In [ ]:
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import io
import tabulate
import helpers_cbc
import helpers_eda_inicial

In [2]:
CBC = '../../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx'

In [ ]:
df_calificaciones, df_carreras = helpers_cbc.get_data(CBC)

In [ ]:
df_calificaciones

In [ ]:
df_calificaciones['Dni'].nunique()

## Vuelvo a revisar estadisticas

In [ ]:
helpers_eda_inicial.initial_eda(df_calificaciones, "df_calificaciones")

In [ ]:
# Miro si la distribución de las variables categoricas

# Variables a analizar
categoricas_principales = ['Carrera', 'dominio email', 'Materia', 'Nota', 'UBA XXI', 'Es materia FCEN?']

# Crear subplots con layout más generoso
fig, axes = plt.subplots(nrows=len(categoricas_principales), ncols=1, figsize=(18, 6 * len(categoricas_principales)), constrained_layout=True)

# Asegurar que axes sea iterable
if len(categoricas_principales) == 1:
    axes = [axes]

for i, col in enumerate(categoricas_principales):
    ax = axes[i]
    
    # Calcular distribución
    ciclo_counts = df_calificaciones[col].value_counts(normalize=True)
    
    # Plot
    ciclo_counts.plot(kind="bar", color="purple", ax=ax, width=0.8)

    # Ajustes visuales
    ax.set_title(f"Distribución de {col}", fontsize=14)
    ax.set_xlabel(col, fontsize=12)
    ax.set_ylabel("Proporción", fontsize=12)
    ax.tick_params(axis='x', rotation=70)  # Rotar etiquetas del eje x
    ax.tick_params(axis='y', labelsize=10)
    
    # Si hay muchas categorías, reducir el tamaño de fuente de las etiquetas del eje X
    if len(ciclo_counts) > 20:
        ax.set_xticklabels(ax.get_xticklabels(), fontsize=8)

plt.show()

Quiero ver cuanta gente no llega a tener 6 valores numericos en las cosas que rindio (primero en general, luego veo en la FCEN)

In [ ]:
df_calificaciones['Nota'].unique()

In [ ]:
# Paso 1: Filtrar notas que son numéricas
notas_numericas = [str(i) for i in range(0, 11)]
df_numericas = df_calificaciones[df_calificaciones['Nota'].isin(notas_numericas)]

# Paso 2: Contar la cantidad de notas numéricas por Dni
conteo = df_numericas.groupby('Dni').size()

# Paso 3: Contar cuántos Dni tienen al menos 6 notas numéricas
resultado = (conteo >= 6).sum()

print(f'Cantidad de personas con al menos 6 notas numéricas: {resultado}')

In [ ]:
notas_numericas

Para aprobar el CBC necesitas tener todas las materias de la FCEN aprobadas, eso es con 4 o más.

In [ ]:
# Paso 1: Filtrar notas que son numéricas
notas_numericas_4_o_mas = [str(i) for i in range(4, 11)]
df_numericas_4_o_mas = df_calificaciones[df_calificaciones['Nota'].isin(notas_numericas_4_o_mas)]

# Paso 2: Contar la cantidad de notas numéricas por Dni
conteo_4_o_mas = df_numericas_4_o_mas.groupby('Dni').size()

# Paso 3: Contar cuántos Dni tienen al menos 6 notas numéricas
resultado = (conteo_4_o_mas >= 6).sum()

print(f'Cantidad de personas con al menos 6 notas numéricas: {resultado}')

Ahora nos ponemos mas finos, necesito que dichas materias sean de la FCEN (hay que tener en cuenta que pueden tener un AP, aprobado por resolución)

In [ ]:
# Paso 1: Filtrar notas que son numéricas
notas_numericas_4_o_mas_FCEN = [str(i) for i in range(4, 11)]
notas_numericas_4_o_mas_FCEN.append('AP')
df_numericas_4_o_mas_FCEN = df_calificaciones[(df_calificaciones['Nota'].isin(notas_numericas_4_o_mas_FCEN)) & (df_calificaciones['Es materia FCEN?']=='SI')]

# Paso 2: Contar la cantidad de notas numéricas por Dni
conteo_4_o_mas_FCEN = df_numericas_4_o_mas_FCEN.groupby('Dni').size()

# Paso 3: Contar cuántos Dni tienen al menos 6 notas numéricas
resultado = (conteo_4_o_mas_FCEN >= 6).sum()

print(f'Cantidad de personas con al menos 6 notas numéricas: {resultado}')

Si relajamos un poco porque tal vez deben un final...

In [ ]:
# Paso 1: Filtrar notas que son numéricas
notas_numericas_4_o_mas_FCEN = [str(i) for i in range(4, 11)]
notas_numericas_4_o_mas_FCEN.append('AP')
df_numericas_4_o_mas_FCEN = df_calificaciones[(df_calificaciones['Nota'].isin(notas_numericas_4_o_mas_FCEN)) & (df_calificaciones['Es materia FCEN?']=='SI')]

# Paso 2: Contar la cantidad de notas numéricas por Dni
conteo_4_o_mas_FCEN = df_numericas_4_o_mas_FCEN.groupby('Dni').size()

# Paso 3: Contar cuántos Dni tienen al menos 6 notas numéricas
resultado = (conteo_4_o_mas_FCEN >= 5).sum()

print(f'Cantidad de personas con al menos 5 notas numéricas: {resultado}')

Mirando un poco los casos que no toma...

In [ ]:
# Paso 3: Obtener los DNIs con al menos 6 notas numéricas
dnis_validos = conteo_4_o_mas_FCEN[conteo_4_o_mas_FCEN >= 6].index

# Paso 4: Filtrar el dataframe original por esos DNIs
df_filtrado = df_calificaciones[df_calificaciones['Dni'].isin(dnis_validos)]

In [ ]:
df_filtrado

Esta persona tiene todo aprobado (pues 'Algebra A' es equivalente a Algebra) pero no figura de esa manera en las condiciones que puse arriba porque ALGEBRA A no es de la FCEN.

In [ ]:
df_calificaciones[~df_calificaciones['Dni'].isin(dnis_validos)][df_calificaciones['Dni'] == 47866196]

Tengo incluso más cuidado. Voy a pedir que las notas sean mayores o iguales a 4 y que sea una materia de la FCEN o se corresponda con ALGEBRA A

In [ ]:
# Paso 1: Filtrar notas que son numéricas
notas_numericas_4_o_mas_FCEN_algebra = [str(i) for i in range(4, 11)]
notas_numericas_4_o_mas_FCEN_algebra.append('AP')
df_numericas_4_o_mas_FCEN_algebra = df_calificaciones[(df_calificaciones['Nota'].isin(notas_numericas_4_o_mas_FCEN)) & ((df_calificaciones['Es materia FCEN?']=='SI' )| (df_calificaciones['Materia'] == 'ALGEBRA A'))]

# Paso 2: Contar la cantidad de notas numéricas por Dni
conteo_4_o_mas_FCEN_algebra = df_numericas_4_o_mas_FCEN_algebra.groupby('Dni').size()

# Paso 3: Contar cuántos Dni tienen al menos 6 notas numéricas
resultado = (conteo_4_o_mas_FCEN_algebra >= 6).sum()

print(f'Cantidad de personas con al menos 6 notas numéricas: {resultado}')

Se acerca mucho más al número que vimos que logramos mergear con la FCEN

In [ ]:
# Paso 3: Obtener los DNIs con al menos 6 notas numéricas
dnis_validos = conteo_4_o_mas_FCEN_algebra[conteo_4_o_mas_FCEN_algebra >= 6].index

# Paso 4: Filtrar el dataframe original por esos DNIs
df_filtrado = df_calificaciones[df_calificaciones['Dni'].isin(dnis_validos)]

In [ ]:
df_calificaciones[~df_calificaciones['Dni'].isin(dnis_validos)]['Dni'].unique()

In [ ]:
df_calificaciones[~df_calificaciones['Dni'].isin(dnis_validos)][df_calificaciones['Dni'] == 43404527]

y si miro la cantidad que tiene 5 materias con esas caracteristicas

In [ ]:
# Paso 1: Filtrar notas que son numéricas
notas_numericas_4_o_mas_FCEN_algebra = [str(i) for i in range(4, 11)]
notas_numericas_4_o_mas_FCEN_algebra.append('AP')
df_numericas_4_o_mas_FCEN_algebra = df_calificaciones[(df_calificaciones['Nota'].isin(notas_numericas_4_o_mas_FCEN)) & ((df_calificaciones['Es materia FCEN?']=='SI' )| (df_calificaciones['Materia'] == 'ALGEBRA A')) & (df_calificaciones['Fecha'].dt.year < 2025)]

# Paso 2: Contar la cantidad de notas numéricas por Dni
conteo_4_o_mas_FCEN_algebra = df_numericas_4_o_mas_FCEN_algebra.groupby('Dni').size()

# Paso 3: Contar cuántos Dni tienen al menos 6 notas numéricas
resultado = (conteo_4_o_mas_FCEN_algebra >= 5).sum()

print(f'Cantidad de personas con al menos 5 notas numéricas: {resultado}')

# Rearmando las columnas

Voy a armar el codigo para conseguir un dataframe que tenga una columna por cluster. Debe tener:

* dni: así luego puedo juntar la información

* fecha_clustex: fecha en que rindió, donde x cambia según el número de cluster

* nota_clusterx: nota que sacó, donde x cambia según el número de cluster

* uba_xxi_clustex: si cursó por UBAXXI o no, donde x cambia según el número de cluster

* cantidad_de_veces_que_rindio_clusterx: cantidad de veces que rindió la materia del cluster x, donde x cambia según el número de cluster. <--Esto se puede hacer si las personas tienen solo una materia de cada cluster (lo voy a chequear)

Sabemos que el cluster 8 se corresponde con todas materias que no son de la FCEN. Si rindió más de una vez una materia decidimos guardar el promedio.

Para el cluster 8 tenemos varias materias, entonces voy a ver si las personas tienen solo una materia de cada cluster y en caso de no ser así podemos poner una columna que sea 'cantidad de materias que hizo que no son de una carrera de la FCEN'.

In [ ]:
df_notas = df_calificaciones[['Dni', 'Materia', 'Fecha', 'Nota', 'UBA XXI']].copy()

Cada estudiante tiene un registro por materia

In [ ]:
# para cada DNI, contar la cantidad de materias repetidas
df_notas.groupby(['Dni', 'Materia']).size().reset_index(name='count')

Tiene sentido tomar los ausentes (A) y los NR (no regularizó) como un 0, pero qué hacemos con los AP (aprobados por resolución)?

In [ ]:
df_notas[df_notas['Nota'] == 'AP']

In [ ]:
df_notas[df_notas['Nota'] == 'AP']['Materia'].value_counts()

Me gustaría ver la distribución de notas por materia

In [ ]:
# Miro si la distribución de las variables categoricas

# Variables a analizar
materias = df_calificaciones[df_calificaciones['Es materia FCEN?'] == 'SI']['Materia'].unique()

# Crear subplots con layout más generoso
fig, axes = plt.subplots(nrows=len(materias), ncols=1, figsize=(18, 6 * len(materias)), constrained_layout=True)

# Asegurar que axes sea iterable
if len(materias) == 1:
    axes = [axes]

for i, col in enumerate(materias):
    ax = axes[i]
    
    # Calcular distribución
    ciclo_counts = df_calificaciones[df_calificaciones['Materia'] == col]['Nota'].value_counts(normalize=True)
    
    # Plot
    ciclo_counts.plot(kind="bar", color="purple", ax=ax, width=0.8)

    # Ajustes visuales
    ax.set_title(f"Distribución de {col}", fontsize=14)
    ax.set_xlabel(col, fontsize=12)
    ax.set_ylabel("Proporción", fontsize=12)
    ax.tick_params(axis='x', rotation=70)  # Rotar etiquetas del eje x
    ax.tick_params(axis='y', labelsize=10)
    
    # Si hay muchas categorías, reducir el tamaño de fuente de las etiquetas del eje X
    if len(ciclo_counts) > 20:
        ax.set_xticklabels(ax.get_xticklabels(), fontsize=8)

plt.show()

Considerando que AP es de lo que menos aparece, podemos no tenerlos en cuenta para el promedio, pero crear una columna que indique si aprobó algo por resulución.

In [ ]:
df_carreras

In [ ]:
# Diccionario con materias agrupadas por cluster (ya normalizadas)
cluster_dict = {
    1: ['BIOLOGIA', 'BIOLOGIA  E INTRODUCCION A LA BIOLOGIA CELULAR'],
    2: ['FISICA'],
    3: ['INT. AL CONOC. DE LA SOCIEDAD Y EL ESTADO'],
    4: ['INT. AL PENSAMIENTO CIENTIFICO'],
    5: ['MATEMATICA 9H', 'ALGEBRA', 'ALGEBRA A', 'ANALISIS MATEMATICO A'],
    6: ['QUIMICA'],
    7: ['INTROD. AL PENSAMIENTO COMPUTACIONAL'],
    8: [
        'SOCIOLOGIA', 'PRINCIPIOS DE DD HH Y DERECHO CONSTITUCIONAL', 'CIENCIA POLITICA',
        'PRINCIPIOS GENERALES DE DERECHO PRIVADO', 'SEMIOLOGIA',
        'INTROD.AL CONOCIMIENTO PROYECTUAL I', 'TALLER DE DIBUJO',
        'ECONOMIA', 'FILOSOFIA', 'PSICOLOGIA', 'HISTORIA ECONOMICA SOCIAL GRAL.',
        'INTROD.AL CONOCIMIENTO PROYECTUAL II', 'ANTROPOLOGIA',
        'HISTORIA ECONOMICA Y SOCIAL GENERAL', 'METODOLOGIA DE LAS CIENCIAS SOCIALES',
        'TRABAJO Y SOCIEDAD', 'PRINCIPIOS GENERALES DEL DERECHO LATINOAMERICANO',
        'FISICA  E INTRODUCCION A LA BIOFISICA', 'MATEMATICA', 'ANALISIS MATEMATICO I',
        'ANALISIS MATEMATICO'
    ]
}

# Invertimos el diccionario para mapear materia -> cluster
materia_a_cluster = {}
for cluster, materias in cluster_dict.items():
    for materia in materias:
        materia_a_cluster[materia] = cluster

# Aplicamos el mapeo al dataframe
df_carreras['Cluster'] = df_carreras['Materia'].apply(lambda x: materia_a_cluster.get(x, None))

Esto mismo, lo puedo hacer para df_calificaciones o df_notas.

In [ ]:
# Aplicamos el mapeo al dataframe
df_calificaciones['Cluster'] = df_calificaciones['Materia'].apply(lambda x: materia_a_cluster.get(x, None))

In [ ]:
df_calificaciones['Cluster'].unique()

In [ ]:
# Paso 1: eliminar filas duplicadas por DNI y Materia
df_unico = df_calificaciones.drop_duplicates(subset=['Dni', 'Materia'])

# Paso 2: agrupar por DNI y Cluster, y contar materias distintas
conteo_materias_distintas = df_unico.groupby(['Dni', 'Cluster']).agg({'Materia': 'nunique'}).reset_index()
conteo_materias_distintas.rename(columns={'Materia': 'Materias_distintas'}, inplace=True)

# Paso 3: filtrar los casos con más de una materia distinta en el mismo cluster
repetidos = conteo_materias_distintas[conteo_materias_distintas['Materias_distintas'] > 1]

# Mostrar el resultado
print(repetidos)

In [ ]:
repetidos['Cluster'].unique()

In [ ]:
# Suponemos que df ya tiene las columnas: DNI, Cluster, Nota
df_calificaciones['Nota'] = df_calificaciones['Nota'].replace('A', '0')
df_calificaciones['Nota'] = df_calificaciones['Nota'].replace('NR', '0')

#elimino filas que tienen AP en la columna 'Nota'
df_calificaciones= df_calificaciones[df_calificaciones['Nota'] != 'AP']

#paso la columna a integer
df_calificaciones['Nota'] = df_calificaciones['Nota'].astype(int)

# Paso 1: eliminar duplicados si un DNI tiene la misma materia varias veces (opcional, según tu caso)
df_unico = df_calificaciones.drop_duplicates(subset=['Dni', 'Materia', 'Cluster'])

# Paso 2: agrupar por DNI y Cluster, y calcular promedio de Nota
promedios = df_unico.groupby(['Dni', 'Cluster'])['Nota'].mean().reset_index()

# Paso 3: pivotear para tener una columna por cluster
df_pivot = promedios.pivot(index='Dni', columns='Cluster', values='Nota')

# (Opcional) Renombrar columnas para que digan "Cluster_1", "Cluster_2", etc.
df_pivot.columns = [f'Cluster_{int(col)}' for col in df_pivot.columns]

# (Opcional) Reset index si querés que el DNI vuelva a ser columna
df_pivot = df_pivot.reset_index()

df_pivot

Para armar una columna sobre las fechas en que rindió cada cluster, puedo trabajar con la tabla que tiene la columna agregada "Cluster" y luego mergearla con la tabla que acabo de crear.